In [8]:
# utility
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# keras tensorflow wrapper
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications import InceptionV3, Xception
from tensorflow.python.keras.models import Sequential,load_model
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras.metrics import top_k_categorical_accuracy
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.optimizers import Adam

# scikit-learn helper function
from sklearn.utils.class_weight import compute_class_weight

In [9]:
def top_3_accuracy(true, pred):
    return top_k_categorical_accuracy(true, pred, k=3)

def path_join(dirname, img_paths):
    return [os.path.join(dirname, img_path) for img_path in img_paths]

In [10]:
def plot_images(images, cls_true, cls_pred):
    
    assert len(images) == len(cls_true)
    if len(images) == 0: return

    num_x_plot = int(np.ceil(len(images)/5))
    fig, axes = plt.subplots(num_x_plot , 5 , figsize=(22,len(images)))
    interpolation = 'spline16'

    fig.subplots_adjust(hspace=0.6)
  
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i], interpolation=interpolation)
            cls_pred_name = cls_pred.argsort(axis=1)[i][-3:][::-1]
            cls_pred_prob = np.sort(cls_pred, axis=1)[i][-3:][::-1]
            cls_true_name = cls_true[i]
            xlabel = ""
            for idx, class_name in enumerate(cls_pred_name):
                xlabel += "Pred {} : {}, score: {:.04f} \n".format(idx, idx_cls[class_name], cls_pred_prob[idx])
                xlabel += "True:{}".format(idx_cls[cls_true_name])
                ax.set_xlabel(xlabel, fontsize=15.0)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.tight_layout()
    plt.show()
    
def load_images(image_paths):
    images = [plt.imread(path) for path in image_paths]
    return np.asarray(images)

def print_errors(class_num, cls_test, cls_pred, size=15):
    generator_test.reset()

    cls_pred_argmax = cls_pred.argmax(axis=1) # 예측 클래스 어래이
    incorrect = (cls_pred_argmax != cls_test) # 틀린 예측 인덱스
    true_incorrect = cls_test[incorrect]
    pred_incorrect = cls_pred_argmax[incorrect]

    pred_idx = np.argwhere(pred_incorrect == class_num).flatten() 

    print('예측 틀린 개수:{}'.format(len(pred_idx)))

    rdm_idx = np.random.randint(0, len(pred_idx), size)
    # 틀린 index 중 주어진 카테고리 번호의 prediction 이 틀린 index 중 랜덤한 size개의 index의 image_path
    image_path = np.array(path_join(test_dir_name, generator_test.filenames))[incorrect][pred_idx][rdm_idx]
    image = load_images(image_path)

    true_incorrect = true_incorrect[pred_idx][rdm_idx]
    pred_incorrect = cls_pred[incorrect][pred_idx][rdm_idx]

    plot_images(images=image, cls_true=true_incorrect, cls_pred=pred_incorrect)
    
    
def print_confusion_matrix(cls_test, cls_pred, labels, labels_actual):
  
    cm = confusion_matrix(y_true=cls_test,
                          y_pred=cls_pred.argmax(axis=1),
                          labels=labels)
    
    true_pos = np.diag(cm)
    true_pos_false_pos = np.sum(cm, axis=0).astype(float)
    true_pos_false_neg = np.sum(cm, axis=1).astype(float)

    precision = true_pos / (true_pos_false_pos)
    recall = true_pos / (true_pos_false_neg)

    # please take care of divison by zero
    precision[np.isnan(precision)] = 0.0
    recall[np.isnan(recall)] = 0.0

    plt.figure(figsize=(20,10))
    sns.heatmap(cm, 
                annot=True, 
                xticklabels=label_actual, 
                yticklabels=label_actual, 
                fmt='g')

    plt.xlabel("predicted label")
    plt.ylabel("actual label")
    
    for i, class_name in enumerate(label_actual):
        print("({0}) {1}: recall={2:.2f}, precision={3:.2f}".format(i, class_name , recall[i], precision[i]))

In [11]:
# 모델 불러오기 
xception = load_model('./checkpoiint/sequential_fe.h5')
# inception = load_model('./model/')

ValueError: Unknown metric function:top_3_accuracy

In [12]:
TEST_PATH = './dataset/test'
datagen_test = ImageDataGenerator(rescale=1./255)

batch_size = 256
input_shape = (224,224)
generator_test = datagen_test.flow_from_directory(directory=TEST_PATH,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False
                                                  )

step_size = generator_test.n / batch_size + 1

# model로 test set 추론
xception_pred = xception.model.predict_generator(generator_test, steps=step_size, verbose=1)
inception_pred = inception.model.predict_generator(generator_test, steps=step_size, verbose=1)

classes = np.sort(generator_test.class_indices.values()) # 카테고리 번호 
classes_actual = np.sort(generator_test.class_indices.keys()) # 카테고리 명

cls_test = generator_test.classes

idx_cls = generator_test.class_indices()

Found 2243 images belonging to 19 classes.


NameError: name 'xception' is not defined